In [ ]:
#Hedging our bets muni-style

In [1]:
#import statements for packages/libraries
#which libraries do we need?
#pyfolio
#scipy
#backtester

import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from dotenv import load_dotenv
import panel as pn
from panel.interact import interact
from panel import widgets
import alpaca_trade_api as tradeapi
import quandl
import os
import requests

ModuleNotFoundError: No module named 'plotly'

In [ ]:
#pull in data from API (7-ish years of data 1/1/2013-11/01/2020)
#bonds--bloomberg
#muni etf (MUB)--alpaca
#swap--bloomberg or alpaca
#treasury--quandl
#treasury ETF (TLT)--alpaca

In [ ]:
load_dotenv()

In [ ]:
#import

api_key_QD = os.getenv("QUANDL_API_KEY")
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# type(api_key_QD)
# type(alpaca_api_key)

In [ ]:
#import alpaca api file for ETF price history
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
start_date = pd.Timestamp("2013-01-02", tz="America/New_York").isoformat()
today = pd.Timestamp("2020-11-02", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["MUB", "TLT", "UDN", "UUP","LQD","JNK"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get closing prices
df_ETFs = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = today
).df

# Preview DataFrame
# YOUR CODE HERE!
df_ETFs.head()

# Output the data to CSV
df_ETFs.to_csv("Resources/ETFs1.csv", encoding='utf-8', index=True)

In [ ]:
ETF_csv = Path("Resources/ETFs.csv")
ETF_df = pd.read_csv(ETF_csv, index_col='Unnamed: 0', infer_datetime_format=True, parse_dates=True)
#ETF_df.rename(columns={'Unnamed: 0': 'Date'}, inplace = True)
ETF_df = ETF_df.drop(ETF_df.index[0])
#ETF_df['Date'] = pd.to_datetime(ETF_df['Date'], utc=True)
ETF_df.index = pd.to_datetime(ETF_df.index, utc=True).date
#ETF_df['Date']= ETF_df['Date'].dt.strftime('%m/%d/%Y')
ETF_df.sort_index(inplace=True)

#drop unneccessary columns and keep close price columns
ETF_close_prices_df = ETF_df.loc[:, ['JNK.3', 'LQD.3', 'MUB.3', 'TLT.3', 'UDN.3', 'UUP.3']]
ETF_close_prices_df.rename(columns={'JNK.3': 'JNK',
                   'LQD.3': 'LQD',
                   'MUB.3': 'MUB',
                   'TLT.3': 'TLT',
                   'UDN.3': 'UDN',
                   'UUP.3': 'UUP',
                  }, inplace=True)

ETF_close_prices_df[['JNK', 'LQD', 'MUB', 'TLT', 'UDN', 'UUP']] = ETF_close_prices_df[['JNK', 'LQD', 'MUB', 'TLT', 'UDN', 'UUP']].astype(float)
ETF_close_prices_df.dtypes

#calculate daily returns
ETF_daily_returns_df = ETF_close_prices_df.pct_change().dropna()
ETF_daily_returns_df.head()

In [ ]:
#import quandl api file for US Treasury Rate history
USTREASURY_data = quandl.get('USTREASURY/YIELD', start_date='2013-01-01', end_date='2020-11-06',authtoken=api_key_QD)
#USTREASURY_data.to_csv('Resources/USTREASURY1.csv', encoding='utf-8', index=True)
USTREASURY_data.head()
columns_to_drop = ['1 MO', '2 MO', '3 MO', '6 MO']
USTREASURY_data.drop(columns=columns_to_drop, inplace=True)
USTREASURY_data.sort_index(inplace=True)

#create new dataframe with maturity values
columns = []
tsy_mty_df = USTREASURY_data.copy()
for column in USTREASURY_data.columns:
    maturity = ''
    for item in column.split():
        if item.isdigit():
            maturity = maturity + item
    #print(int(maturity[0]))
    tsy_mty_df[column] = int(maturity)
    columns.append(maturity)
USTREASURY_data.columns = columns
tsy_mty_df.columns = columns

#USTREASURY_data.head()
#tsy_mty_df.head()

#define function for calculating Treasury bond prices
def bondprice(fv, c, ytm, t, m):
    bondprice = ((fv*c/m*(1-(1+ytm/m)**(-m*t)))/(ytm/m)) + fv*(1+(ytm/m))**(-m*t)
    return(bondprice)

#bondprice(1000,0.06,0.08,9,2)
#bondprice(100,0.0015,0.0014,1,2)

#create new dataframe with treasury bond prices
fv = 100
c = USTREASURY_data.shift(1)/100
ytm = USTREASURY_data/100
m = 2
t = tsy_mty_df
USTREASURY_daily_prices = bondprice(fv, c, ytm, t, m)

USTREASURY_data_daily_returns = USTREASURY_daily_prices.pct_change().dropna()
#USTREASURY_data_daily_returns.head()
USTREASURY_data_daily_returns.columns = USTREASURY_data_daily_returns.columns.astype(int)
USTREASURY_data_daily_returns.head()
#tsy_mty_df.head()

In [ ]:
# pull in LIBOR swaps to create pct_change df
## the sort_index for dates doesn't seem to be working...
swaps_csv = Path("Resources/libor_swaps.csv")
swaps_df = pd.read_csv(swaps_csv, index_col='Date', infer_datetime_format=True, parse_dates=True)
#swaps_df.set_index('Date',inplace=True)
swaps_df.sort_index(inplace=True)

swaps_mty_df = swaps_df.copy()
for column in swaps_df.columns:
    swaps_mty_df[column] = int(column)

#create new dataframe with libor swap value (using bond price function)
fv = 100
c = swaps_df.shift(1)/100
ytm = swaps_df/100
m = 2
t = swaps_mty_df
swaps_df_values = bondprice(fv, c, ytm, t, m)

swaps_daily_returns = swaps_df_values.pct_change().dropna()
swaps_daily_returns.columns = swaps_daily_returns.columns.astype(int)
swaps_daily_returns.head()

In [ ]:
#pull in bond prices and create dataframe
bonds_csv = Path("Resources/bonds_example_prices_larger.csv")
bonds_df = pd.read_csv(bonds_csv, index_col='Date',infer_datetime_format=True, parse_dates=True)
bonds_df.sort_index(inplace=True)

#need to wait to dropna after concat within the correlation function
bonds_daily_returns = bonds_df.pct_change()
bonds_daily_returns.head()

In [ ]:
#pull in bond descriptions to dataframe
bondescr_csv = Path("Resources/muni_hedge_characteristics.csv")
bondescr_df = pd.read_csv(bondescr_csv)

cleaned_cusips = []
for cusipid in bondescr_df['CUSIP']:
    cusipid = cusipid.strip()
    cleaned_cusips.append(cusipid)

bondescr_df['CUSIP'] = cleaned_cusips

bondescr_df.set_index('CUSIP',inplace=True)
bondescr_df.head()

In [ ]:
#function passing a cusip to put it against hedges
#what are the steps we need our function to do:
    #1.pick bond
    #2. compare versus all hedges
    #3. show correlation of bond vis a vis different hedges
#ETF_daily_returns_df, USTREASURY_data_daily_returns, swaps_daily_returns, bonds_daily_returns  
#cusip = '57583R5Q4'
def historical_corr(cusip):
    # find maturity of chosen muni bond and convert to datetime
    bond_mty = bondescr_df.loc[cusip,'Maturity Date']
    bond_mty = dt.datetime.strptime(bond_mty, '%m/%d/%Y')
    bond_mty

    # create series with bond years to maturity for lookup in dataframes
    # relativedelta function: https://dateutil.readthedocs.io/en/stable/relativedelta.html
    ## make output for years more exact by looking at days/360 and rounding?
    mty_years = []
    for return_date in bonds_daily_returns.index:
        difference = relativedelta(bond_mty, return_date)
        mty_years.append(difference.years)
    #len(mty_years)

    # reference closest maturity in tsy and swap df
    # USTREASURY_data_daily_returns, swaps_daily_returns, mty_years
    tsy_index = []
    for item in mty_years:
        min_item = ' '
        min_value = 0
        for col in USTREASURY_data_daily_returns.columns:
            if min_item == ' ':
                min_item = col
                min_value = abs(col - int(item))
            elif abs(col - int(item)) < min_value:
                min_item = col
                min_value = abs(col - int(item))
        tsy_index.append(min_item)

    swap_index = []
    for item in mty_years:
        min_item = ' '
        min_value = 0
        for col in swaps_daily_returns.columns:
            if min_item == ' ':
                min_item = col
                min_value = abs(col - int(item))
            elif abs(col - int(item)) < min_value:
                min_item = col
                min_value = abs(col - int(item))
        swap_index.append(min_item)
    #len(swap_index)
    #len(tsy_index)

    # create new dataframe to combine items
    # USTREASURY_data_daily_returns, swaps_daily_returns, mty_years
    joined_df = pd.DataFrame(bonds_daily_returns[cusip])
    joined_df['tsy_index'] = tsy_index
    joined_df['swap_index'] = swap_index
    
    joined_df = pd.merge(joined_df, USTREASURY_data_daily_returns, left_index=True, right_index=True)

    tsy_change = []
    for indx in joined_df.index:
        change = joined_df.loc[indx, joined_df['tsy_index'][indx]]
        tsy_change.append(change)
    joined_df['tsy_change'] = tsy_change
    tsy_col_drop = ['tsy_index',1,2,3,5,7,10,20,30]
    joined_df.drop(columns=tsy_col_drop,inplace=True)

    joined_df = pd.merge(joined_df, swaps_daily_returns, left_index=True, right_index=True)

    swap_change = []
    for indx in joined_df.index:
        change = joined_df.loc[indx, joined_df['swap_index'][indx]]
        swap_change.append(change)
    joined_df['swap_change'] = swap_change
    swap_col_drop = ['swap_index',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
    joined_df.drop(columns=swap_col_drop,inplace=True)

    ##concat separate dataframes (muni etf, treasury etf)
    #ETF_daily_returns_df
    joined_df = pd.merge(joined_df, ETF_daily_returns_df, left_index=True, right_index=True)
    #joined_df.head()

    ## calculate standard deviation
    hedge_corr = joined_df.corr()
    return hedge_corr
hedge_corr = historical_corr(cusip)
hedge_corr

In [ ]:
hedge_corr.style.background_gradient(cmap='YlGn_r')

In [ ]:
## heatmap
import seaborn as sns
sns.heatmap(hedge_corr, vmin=-1, vmax=1)

In [ ]:
def historical_corr_test(cusip):
    # find maturity of chosen muni bond and convert to datetime
    bond_mty = bondescr_df.loc[cusip,'Maturity Date']
    bond_mty = dt.datetime.strptime(bond_mty, '%m/%d/%Y')
    bond_mty

    # create series with bond years to maturity for lookup in dataframes
    # relativedelta function: https://dateutil.readthedocs.io/en/stable/relativedelta.html
    ## make output for years more exact by looking at days/360 and rounding?
    mty_years = []
    for return_date in bonds_daily_returns.index:
        difference = relativedelta(bond_mty, return_date)
        mty_years.append(difference.years)
    #len(mty_years)

    # reference closest maturity in tsy and swap df
    # USTREASURY_data_daily_returns, swaps_daily_returns, mty_years
    tsy_index = []
    for item in mty_years:
        min_item = ' '
        min_value = 0
        for col in USTREASURY_data_daily_returns.columns:
            if min_item == ' ':
                min_item = col
                min_value = abs(col - int(item))
            elif abs(col - int(item)) < min_value:
                min_item = col
                min_value = abs(col - int(item))
        tsy_index.append(min_item)

    swap_index = []
    for item in mty_years:
        min_item = ' '
        min_value = 0
        for col in swaps_daily_returns.columns:
            if min_item == ' ':
                min_item = col
                min_value = abs(col - int(item))
            elif abs(col - int(item)) < min_value:
                min_item = col
                min_value = abs(col - int(item))
        swap_index.append(min_item)
    #len(swap_index)
    #len(tsy_index)

    # create new dataframe to combine items
    # USTREASURY_data_daily_returns, swaps_daily_returns, mty_years
    joined_df = pd.DataFrame(bonds_daily_returns[cusip])
    joined_df['tsy_index'] = tsy_index
    joined_df['swap_index'] = swap_index
    
    joined_df = pd.merge(joined_df, USTREASURY_data_daily_returns, left_index=True, right_index=True)

    tsy_change = []
    for indx in joined_df.index:
        change = joined_df.loc[indx, joined_df['tsy_index'][indx]]
        tsy_change.append(change)
    joined_df['tsy_change'] = tsy_change
    tsy_col_drop = ['tsy_index',1,2,3,5,7,10,20,30]
    joined_df.drop(columns=tsy_col_drop,inplace=True)

    joined_df = pd.merge(joined_df, swaps_daily_returns, left_index=True, right_index=True)

    swap_change = []
    for indx in joined_df.index:
        change = joined_df.loc[indx, joined_df['swap_index'][indx]]
        swap_change.append(change)
    joined_df['swap_change'] = swap_change
    swap_col_drop = ['swap_index',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
    joined_df.drop(columns=swap_col_drop,inplace=True)

    ##concat separate dataframes (muni etf, treasury etf)
    #ETF_daily_returns_df
    joined_df = pd.merge(joined_df, ETF_daily_returns_df, left_index=True, right_index=True)
    #joined_df.head()

    ## calculate standard deviation
    hedge_corr = joined_df.corr()
    return hedge_corr.iloc[1,0]

In [2]:
tsy_corr = []
for item in bonds_daily_returns.columns:
    cusip = item
    tsy_corr.append(historical_corr_test(cusip))
tsy_corr

NameError: name 'bonds_daily_returns' is not defined